# Aave Proposal Pivotality

In [1]:
# sets up the pynb environment
from datetime import datetime
from zoneinfo import ZoneInfo
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

In [2]:
print(f"Last updated on {datetime.now(ZoneInfo('UTC')).date()}")

Last updated on 2023-04-20


This article is part of a series of reports that aims to determine the impact of large token holders (whales) on DAO governance.

We take a look at the top 10 Aave Snapshot proposals ranked by voting power and voter count, including the first Butter Aave Elections.

We compare their voter turnout and pivotality metric (a measure of whale influence, described in our previous [Mirror article](https://mirror.xyz/butterd.eth/_VrHsBXZU0IYuIUMq0ifJTffvH0e1AKNh6I-BIV3Nc4) and [data report](https://butterymoney.github.io/gov_analysis/report/plutocracy_report.html)).

In [3]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [4]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [5]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([id for id, _ in items]), name="Proposal"
            ),
            columns=[
                "Title",
                "Date Started",
                "Date Ended",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "Result",
                "Result w/o Whales"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["total_vp"], ascending=False)

        changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]

        score_differences_dfs[organization].drop("outcome_changed", axis=1, inplace=True)

        score_differences_dfs[organization]["Date Started"] = pd.to_datetime(score_differences_dfs[organization]["Date Started"], utc=True, unit="s").dt.strftime("%d/%m/%y")
        score_differences_dfs[organization]["Date Ended"] = pd.to_datetime(score_differences_dfs[organization]["Date Ended"], utc=True, unit="s").dt.strftime("%d/%m/%y")

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization]["Title"] = score_differences_dfs[organization].apply(
            lambda row: f'<a href=http://snapshot.org/#/{space_id}/proposal/{row.name} rel="noopener noreferrer" target="_blank">{row["Title"]}</a>',
            axis=1
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

In [7]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


## Synthesis

Show the percentage of proposals whose outcome changes if whales did not vote (i.e. *whale pivotality*).

In [8]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
Aave,1438,42586,9.33%


---

## Comparison of the top 10 proposals

In [9]:
score_differences_dfs["Aave"] = score_differences_dfs["Aave"].iloc[:10]
HTML(score_differences_dfs["Aave"].rename(lambda index: score_differences_dfs["Aave"].loc[index]["Title"]).drop(["score_differences", "total_vp", "whale_vp_proportion", "Title"], axis=1).to_html(render_links=True, escape=False))

,Date Started,Date Ended,Result,Result w/o Whales
Proposal,,,,
[ARC] Gauntlet <> Aave Renewal,16/11/22,23/11/22,NAY,YAE
[ARC] Updated: Gauntlet <> Aave Renewal,16/12/22,19/12/22,YAE,YAE
[ARC] Community Risk Preference on Aave V3 Borrow/Supply Caps,15/02/23,21/02/23,Aggressive,Conservative only for small mkts
[ARC]- Chaos Labs Risk Parameter Updates - Aave V3 Ethereum - 2023.02.22,28/02/23,03/03/23,YAE,YAE
Supply and Borrow Cap Updates - Aave V3 Avalanche,02/03/23,05/03/23,Aggressive,Aggressive
[TEMP CHECK] Incentivized Delegate Campaign (3-month),03/04/23,10/04/23,TokenLogic,Blockworks Research
Safety Module upgrade to v1.5,15/03/23,20/03/23,YES,YES
[ARC] Community Preference for V3 Migration Levers,27/03/23,05/04/23,NO decrease V2 LT/LTV,NO decrease V2 LT/LTV
Deploy Aave V3 to Metis Andromeda Mainnet,17/03/23,23/03/23,YAE,YAE


### Proportion of Outcomes Changed

In [10]:
print(f"{changed_outcome_proportions['Aave']} of Aave's proposal outcomes change after filtering out whale voting power.")

9.33% of Aave's proposal outcomes change after filtering out whale voting power.


Case Studies
---

#### [ARC] Gauntlet <> Aave Renewal	

**Proposal ID**: [0x549fa21](https://snapshot.org/#/aave.eth/proposal/0x549fa2186f321a4e8a07e0d5e82f85f3e2c83189cfaee678a6df144bb67d8f54)

**Start Date**: 16/11/22
**End Date**: 23/11/22

**Voting System**: Single Choice Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)

This ARC proposal is for the renewal of Gauntlet's 12-month engagement on Aave as a risk manager. This proposal did not pass, but would have passed if whales had not participated.

In [11]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x549fa2186f321a4e8a07e0d5e82f85f3e2c83189cfaee678a6df144bb67d8f54'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x549fa21" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x549fa2186f321a4e8a07e0d5e82f85f3e2c83189cfaee678a6df144bb67d8f54'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAY,ABSTAIN
Scores,477505.187812,639091.170223,9.932691
Whale-only Scores,477179.561740,639076.325697,5.758735
Non-whale Scores,325.626072,14.844526,4.173956


#### [ARC] Updated: Gauntlet <> Aave Renewal	

**Proposal ID**: [0xd096f98](https://snapshot.org/#/aave.eth/proposal/0xd096f98237c642614fe154844c5037a85c1f287f3323c4b003a83bd3f4ea658a)

**Start Date**: 15/12/22
**End Date**: 19/12/22

**Voting System**: Single Choice Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is an updated proposal from the previous proposal which did not pass. It's outcome did not change after filtering out whales.

In [12]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xd096f98237c642614fe154844c5037a85c1f287f3323c4b003a83bd3f4ea658a'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xd096f98" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xd096f98237c642614fe154844c5037a85c1f287f3323c4b003a83bd3f4ea658a'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,ABSTAIN,NAY
Scores,810968.966804,38.584084,13.109700
Whale-only Scores,810686.317376,17.505492,10.953800
Non-whale Scores,282.649428,21.078592,2.155901


#### [ARC] Community Risk Preference on Aave V3 Borrow/Supply Caps

**Proposal ID**: [0xb4c756a](https://snapshot.org/#/aave.eth/proposal/0xb4c756ac701b5cbc0ec2d832652e4f1c820aa40f7d9be18eb81f87080a52c5f1)

**Start Date**: 15/02/23
**End Date**: 21/02/23

**Voting System**: Single Choice Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



A proposal to set supply and borrow cap parameters on non-Ethereum chains. Filtering out whales changed the outcome of this proposal from setting aggressive parameters to only setting conservative parameters for small market cap assets.

In [13]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xb4c756ac701b5cbc0ec2d832652e4f1c820aa40f7d9be18eb81f87080a52c5f1'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xb4c756a" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xb4c756ac701b5cbc0ec2d832652e4f1c820aa40f7d9be18eb81f87080a52c5f1'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,Conservative,Aggressive,Conservative only for small mkts,Abstain,Nay
Scores,1748.206533,492177.899982,305646.846008,45.704975,160.376733
Whale-only Scores,1578.218473,492126.107082,305006.709358,37.704366,153.841623
Non-whale Scores,169.988060,51.792900,640.136650,8.000609,6.535110


#### [ARC]- Chaos Labs Risk Parameter Updates - Aave V3 Ethereum - 2023.02.22

**Proposal ID**: [0xa3f5b45](https://snapshot.org/#/aave.eth/proposal/0xa3f5b45c0238ae1363eb50ffbd59dd19b07c2ce52b7afd21179d6e1416a8ed4c)

**Start Date**: 28/02/23
**End Date**: 03/03/23

**Voting System**: Single Choice Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This proposal is for the adjustment of four risk parameters for USDC and DAI on Aave V3 assets. The outcome of this proposal did not change after filtering out whales.

In [14]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xa3f5b45c0238ae1363eb50ffbd59dd19b07c2ce52b7afd21179d6e1416a8ed4c'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xa3f5b45" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xa3f5b45c0238ae1363eb50ffbd59dd19b07c2ce52b7afd21179d6e1416a8ed4c'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAE,Abstain
Scores,794857.573876,17.605791,12.161327
Whale-only Scores,752869.995664,15.597452,9.350240
Non-whale Scores,41987.578212,2.008339,2.811087


#### Supply and Borrow Cap Updates - Aave V3 Avalanche

**Proposal ID**: [0x4bbd4dc](https://snapshot.org/#/aave.eth/proposal/0x4bbd4dc6928b3d341381c5ba25cf9dfa41386229706832c627aef300963f5864)

**Start Date**: 02/03/23
**End Date**: 05/03/23

**Voting System**: Single Choice Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This proposal aims to adjust twenty supply and borrow caps for eleven Aave V3 assets on Avalanche. The outcome of this proposal did not change after filtering out whales.

In [15]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x4bbd4dc6928b3d341381c5ba25cf9dfa41386229706832c627aef300963f5864'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x4bbd4dc" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x4bbd4dc6928b3d341381c5ba25cf9dfa41386229706832c627aef300963f5864'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,Conservative,Aggressive,NAY,Abstain
Scores,230.814430,749548.068606,6.004477,220.980583
Whale-only Scores,122.780886,470152.599102,3.547180,64.500585
Non-whale Scores,108.033544,279395.469504,2.457296,156.479998


#### [TEMP CHECK] Incentivized Delegate Campaign (3-month)

Aka the first Butter'd Aave Election.

**Proposal ID**: [0xf6da417](https://snapshot.org/#/aave.eth/proposal/0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0)

**Start Date**: 03/04/23
**End Date**: 10/04/23

**Voting System**: Weighted Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



Aave votes to select a delegate who will receive a $15k grant (paid in AAVE) from Aave Grants after a three-month term as a delegate, a delegate election. TokenLogic ultimately won this election, however, if we remove whale voting power from this proposal we find that Blockworks Research would've been selected by the Aave community instead.

In [16]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xf6da417" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,Blockworks Research,ConsenSys,Curia,DAOStewards,Fire Eyes,Flipside Crypto,FranklinDAO,OnChainCoop,Oxytocin,Saludiego201.eth,StableLab,TokenLogic,Wallfacer Labs
Scores,87232.635594,17620.565593,12742.462405,434.508563,104834.493934,90345.343934,66127.639535,409.610873,30504.177435,437.382397,146246.026838,185171.775039,435.859890
Whale-only Scores,86487.989188,16950.926291,12712.705752,407.073261,104802.547664,90318.258696,65886.207100,391.862324,30491.294636,393.737487,146194.584615,185153.921653,420.744217
Non-whale Scores,744.646406,669.639302,29.756653,27.435302,31.946270,27.085238,241.432435,17.748550,12.882799,43.644910,51.442223,17.853386,15.115673


#### Safety Module upgrade to v1.5

**Proposal ID**: [0x743ae60](https://snapshot.org/#/aave.eth/proposal/0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6)

**Start Date**: 15/03/23
**End Date**: 20/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)

A proposal to gauge the community's sentiments on upgrading the Safety Module smart contracts (stkAAVE, stkABPT) to v1.5. The outcome of this proposal did not change after filtering out whales.

In [17]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x743ae60" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YES,NO,ABSTAIN
Scores,728927.638707,34.541017,32.507649
Whale-only Scores,572570.468097,31.963758,3.550356
Non-whale Scores,156357.170609,2.577259,28.957293


#### [ARC] Community Preference for V3 Migration Levers

**Proposal ID**: [0x2eca177](https://snapshot.org/#/aave.eth/proposal/0x2eca177d50449f541586d4816d27022b73417952f8c5996d99f500cf0c7ec1c3)

**Start Date**: 27/03/23
**End Date**: 05/04/23

**Voting System**: Approval Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



The analysis for this proposal is not yet ready to be published.

In [18]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x2eca177d50449f541586d4816d27022b73417952f8c5996d99f500cf0c7ec1c3'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x2eca177" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x2eca177d50449f541586d4816d27022b73417952f8c5996d99f500cf0c7ec1c3'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score_diff - score] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)


#### Deploy Aave V3 to Metis Andromeda Mainnet

**Proposal ID**: [0x7e6175c](https://snapshot.org/#/aave.eth/proposal/0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9)

**Start Date**: 17/03/23
**End Date**: 23/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is a proposal to gauge the community's sentiments to deploy Aave V3 to the Metis "Andromeda" Mainnet. The outcome of this proposal did not change after filtering out whales.

In [19]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x7e6175c" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YAE,NAY,ABSTAIN
Scores,555698.763981,162135.341400,140.725517
Whale-only Scores,133249.462649,161953.210564,137.650972
Non-whale Scores,422449.301332,182.130836,3.074545


#### Chaos Labs Risk Parameter Updates - Aave V3 Arbitrum - 2023.02.20

**Proposal ID**: [0x7fa9944](https://snapshot.org/#/aave.eth/proposal/0x7fa9944e6866ab6dafcebdf1536655faf5a5ba0adc36e6df467702f7ae096c17)

**Start Date**: 25/02/23
**End Date**: 01/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is a proposal to adjust all nine risk parameters for four Aave V3 assets on Arbitrum. The outcome of this proposal did not change after filtering out whales. 

In [20]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7fa9944e6866ab6dafcebdf1536655faf5a5ba0adc36e6df467702f7ae096c17'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x7fa9944" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7fa9944e6866ab6dafcebdf1536655faf5a5ba0adc36e6df467702f7ae096c17'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YAE,NAE,Abastain
Scores,687137.995655,237.064747,1126.592836
Whale-only Scores,686603.505151,222.546671,1118.761185
Non-whale Scores,534.490504,14.518076,7.831651
